In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from qiskit.tools.visualization import plot_bloch_vector
from qiskit import BasicAer
from qiskit.aqua.algorithms import AmplitudeEstimation, MaximumLikelihoodAmplitudeEstimation
from qiskit.aqua.algorithms.single_sample.amplitude_estimation.q_factory import QFactory
from qiskit.aqua.components.uncertainty_problems import UncertaintyProblem

In [2]:
class SineIntegralAFactory(UncertaintyProblem):
    def __init__(self, num_qubits):
        super().__init__(num_qubits + 1)
        self._i_objective = num_qubits

    def build(self, qc, q, q_ancillas=None):
        n = self.num_target_qubits - 1
        q_state = [q[i] for i in range(self.num_target_qubits) if i != self._i_objective]
        q_objective = q[self._i_objective]

        # prepare 1/sqrt{2^n} sum_x |x>_n
        for q_i in q_state:
            qc.h(q_i)

        # apply the sine/cosine term
        qc.ry(2 * 1 / 2 / 2**n, q_objective)

        for i, q_i in enumerate(q_state):
            qc.cry(2 * 2**i / 2**n, q_i, q_objective)


In [3]:
class BernoulliAFactory(UncertaintyProblem):
    """
    Circuit Factory representing the operator A.
    A is used to initialize the state as well as to construct Q.
    """
    
    def __init__(self, probability=0.5):
        super().__init__(1)
        self._probability = probability
        self.i_state = 0
        self._theta_p = 2 * np.arcsin(np.sqrt(probability))
    
    def build(self, qc, q, q_ancillas=None):
        # A is a rotation of angle theta_p around the Y-axis
        qc.ry(self._theta_p, q[self.i_state])


class BernoulliQFactory(QFactory):
    """
    Circuit Factory representing the operator Q.
    This implementation exploits the fact that powers of Q can be implemented efficiently by just multiplying the angle.
    (amplitude estimation only requires controlled powers of Q, thus, only this method is overridden.)
    """
    
    def __init__(self, bernoulli_expected_value):
        super().__init__(bernoulli_expected_value, i_objective=0)
    
    def build(self, qc, q, q_ancillas=None):
        i_state = self.a_factory.i_state
        theta_p = self.a_factory._theta_p
        # Q is a rotation of angle 2*theta_p around the Y-axis
        qc.ry(2 * theta_p, q[i_state])
        
    def build_power(self, qc, q, power, q_ancillas=None):
        i_state = self.a_factory.i_state
        theta_p = self.a_factory._theta_p
        qc.ry(2 * power * theta_p, q[i_state])
    
    def build_controlled_power(self, qc, q, q_control, power, q_ancillas=None, use_basis_gates=True):
        i_state = self.a_factory.i_state
        theta_p = self.a_factory._theta_p
        qc.cry(2 * power * theta_p, q_control, q[i_state])

In [4]:
def get_bernoullis(p):
    a_factory = BernoulliAFactory(p)
    q_factory = BernoulliQFactory(a_factory)
    return a_factory, q_factory

backend = BasicAer.get_backend('qasm_simulator')

In [5]:
ae = MaximumLikelihoodAmplitudeEstimation(3, *get_bernoullis(0.0))
result = ae.run(backend)
print(result['estimation'])
print(ae.confidence_interval(0.05, kind='lr'))

1e-30
[1e-30, 1.600340544152679e-05]


In [10]:
integral = SineIntegralAFactory(3)
backend = BasicAer.get_backend('statevector_simulator')
ae = MaximumLikelihoodAmplitudeEstimation(3, a_factory=integral)

In [18]:
q = QFactory(integral, 3)
print(q.required_ancillas_controlled())

4


In [11]:
circs = ae.construct_circuits()

In [12]:
circs[2].draw()

In [13]:
result = ae.run(backend)
print(result['estimation'])
print(ae.confidence_interval(0.05, kind='lr'))

0.2720783397764194
[0.2720783397764194, 0.2720783397764194]
